In [32]:
KAGGLE=False

In [48]:
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn
import sklearn.preprocessing

from sklearn.pipeline import make_pipeline 
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, OneHotEncoder
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, StratifiedKFold,KFold, cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression


import catboost as cb
from xgboost import XGBRegressor

import pickle
from sklearn.metrics import r2_score

In [34]:
def load_data(tree=None):
    if KAGGLE:
        train = pd.read_csv('data/train.csv')
        test = pd.read_csv('data/test.csv')
        num_train=np.shape(train)[0]
        num_test=np.shape(test)[0]
        alldata=train.append(test, ignore_index=True)
    else:
        alldata=pd.read_csv('./data/Ames_HousePriceLocs.csv')#, index_col=0) #already cleaned (see work in appendix files)
    return alldata

alldata=load_data()

In [35]:
def label_encode():
    from sklearn import preprocessing
    le = preprocessing.LabelEncoder()
    for column_name in alldata.columns:
        if alldata[column_name].dtype == object:
            #cat_idxs.append(alldata.columns.get_loc(column_name))
            #alldata[column_name] = alldata[column_name].astype(str)
            alldata[column_name] = le.fit_transform(alldata[column_name])
label_encode()

In [36]:
def split_into_train_test(alldata):
    y=alldata['SalePrice']
    X=alldata.drop(['SalePrice'],axis=1)
    from sklearn.model_selection import train_test_split
    if KAGGLE:
        return split_train_test(alldata=alldata,num_train=num_train)
    else:
        return train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_test, y_train, y_test = split_into_train_test(alldata) 

In [41]:
from catboost import CatBoostRegressor

from_file = CatBoostRegressor()

cbm=from_file.load_model('./models/model_cb_locs2')

rf=pickle.load(open('./models/model_rf_locs', 'rb'))
xgb=pickle.load(open('./models/model_xgb_locs', 'rb'))

In [71]:
def blended_preds(X):
    return (#(0.1 * rf.predict(X)) + \
            (0.7 * xgb.best_estimator_.predict(X)) + \
            (0.3 * cbm.predict(X))
           )

In [72]:
blended_predictions=blended_preds(X_test)

In [73]:
rmse = np.sqrt(mean_squared_error(y_test,blended_predictions))
r2=r2_score(y_test, blended_predictions)

In [74]:
print(f'RMSE on Test: {rmse}')
print(f'R2 on Test: {r2}')

RMSE on Test: 18265.213193225965
R2 on Test: 0.9484964415569249


The R2 on the Test set is about .95, similar to the R2's of the two strongest predictors